# Data Aggregation and Group Operations

## GroupBy Mechanics

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
....: 'key2' : ['one', 'two', 'one', 'two', 'one'],
....: 'data1' : np.random.randn(5),
....: 'data2' : np.random.randn(5)})

In [4]:
df

,key1,key2,data1,data2
0,a,one,-0.714531,-0.512463
1,a,two,0.732534,1.472051
2,b,one,-1.307308,0.084350
3,b,two,0.460490,0.114843
4,a,one,-0.041879,0.374153


In [5]:
grouped=df['data1'].groupby(df['key1'])

In [6]:
grouped

- this object has all of the information needed to then apply some operation to each of the groups

In [7]:
grouped.mean()

key1
a   -0.007959
b   -0.423409
Name: data1, dtype: float64

In [8]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [9]:
means

key1  key2
a     one    -0.378205
      two     0.732534
b     one    -1.307308
      two     0.460490
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,two
key1,,
a,-0.378205,0.732534
b,-1.307308,0.460490


In [11]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean()

California  2005    0.732534
            2006   -1.307308
Ohio        2005   -0.127021
            2006   -0.041879
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.007959,0.444580
b,-0.423409,0.099597


- Because df['keys'] is not numeric data, so thee is no key2 column in the results

In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.378205 -0.069155
     two   0.732534  1.472051
b    one  -1.307308  0.084350
     two   0.460490  0.114843

### Iterating Over Groups

In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.714531 -0.512463
1    a  two  0.732534  1.472051
4    a  one -0.041879  0.374153
b
  key1 key2     data1     data2
2    b  one -1.307308  0.084350
3    b  two  0.460490  0.114843


In [17]:
pieces = dict(list(df.groupby('key1')))

In [18]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.307308,0.084350
3,b,two,0.460490,0.114843


In [19]:
grouped = df.groupby(df.dtypes, axis=1)

In [22]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.714531 -0.512463
1  0.732534  1.472051
2 -1.307308  0.084350
3  0.460490  0.114843
4 -0.041879  0.374153
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns
- Especially for large datasets, it may be desirable to aggregate only a few columns.

In [23]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.069155
     two   1.472051
b    one   0.084350
     two   0.114843

### Grouping with Dicts and Series

In [24]:
people = pd.DataFrame(np.random.randn(5, 5),
....: columns=['a', 'b', 'c', 'd', 'e'],
....: index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [26]:
people

,a,b,c,d,e
Joe,-1.238034,-0.147354,-0.950131,0.379813,0.415508
Steve,-0.309078,-0.512684,-0.114026,0.132443,0.288238
Wes,0.817814,0.838201,1.025819,0.096841,-1.037753
Jim,1.014487,-0.694531,-0.060702,0.043600,0.009948
Travis,-1.053080,0.492834,-1.322869,0.726218,-1.814407


In [28]:
import numpy as np

In [29]:
people.iloc[2:3, [1,2]]=np.nan

In [30]:
people

,a,b,c,d,e
Joe,-1.238034,-0.147354,-0.950131,0.379813,0.415508
Steve,-0.309078,-0.512684,-0.114026,0.132443,0.288238
Wes,0.817814,NaN,NaN,0.096841,-1.037753
Jim,1.014487,-0.694531,-0.060702,0.043600,0.009948
Travis,-1.053080,0.492834,-1.322869,0.726218,-1.814407


In [31]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
....: 'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [32]:
by_column=people.groupby(mapping, axis=1)

In [33]:
by_column.sum()

,blue,red
Joe,-0.570318,-0.969881
Steve,0.018417,-0.533524
Wes,0.096841,-0.219939
Jim,-0.017102,0.329903
Travis,-0.596651,-2.374653


In [34]:
map_series = pd.Series(mapping)

In [35]:
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,-0.570318,-0.969881
Steve,0.018417,-0.533524
Wes,0.096841,-0.219939
Jim,-0.017102,0.329903
Travis,-0.596651,-2.374653


### Grouping with Functions

In [36]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.594267,-0.841886,-1.010833,0.520254,-0.612298
5,-0.309078,-0.512684,-0.114026,0.132443,0.288238
6,-1.053080,0.492834,-1.322869,0.726218,-1.814407


In [37]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [38]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.238034 -0.147354 -0.950131  0.096841 -1.037753
  two  1.014487 -0.694531 -0.060702  0.043600  0.009948
5 one -0.309078 -0.512684 -0.114026  0.132443  0.288238
6 two -1.053080  0.492834 -1.322869  0.726218 -1.814407

### Grouping by Index Levels

In [39]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
....: [1, 3, 5, 1, 3]],
....: names=['cty', 'tenor'])

In [40]:
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [41]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [42]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.716035 -0.246511 -0.756625  0.529295  1.432417
1     -1.370634 -0.287743  0.700600 -1.501505  1.880077
2      0.251741  0.775102  0.599714  0.726279 -0.198341
3     -0.365096  0.952216 -0.452314  0.185123  0.449489

In [43]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

### Column-Wise and Multiple Function Application

In [44]:
tips=pd.read_csv('examples/tips.csv')

In [45]:
tips['tip_pct']=tips['tip'] / tips['total_bill']

In [47]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


- you may want to aggregate using a different function depending on the column, or multiple functions at once.

In [48]:
grouped = tips.groupby(['day', 'smoker'])

In [49]:
grouped_pct = grouped['tip_pct']

In [50]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [51]:
grouped_pct.agg(['mean', 'std'])

mean       std
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [52]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [53]:
functions = ['count', 'mean', 'max']

In [54]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

<ipython-input-54-426117f1e146>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


In [55]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [59]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [60]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
....: 'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [61]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

In [62]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [63]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [64]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

- The top function is called on each row group from the DataFrame, and then the results are glued together using pandas.concat, labeling the pieces with the group names.

In [65]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [67]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Suppressing the Group Keys

In [68]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [69]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### Quantile and Bucket Analysis

In [70]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
....: 'data2': np.random.randn(1000)})

In [71]:
quartiles=pd.cut(frame.data1,4)

In [72]:
quartiles[:10]

0       (1.54, 3.034]
1    (-1.447, 0.0465]
2      (0.0465, 1.54]
3    (-1.447, 0.0465]
4      (0.0465, 1.54]
5      (0.0465, 1.54]
6      (0.0465, 1.54]
7      (0.0465, 1.54]
8      (0.0465, 1.54]
9      (0.0465, 1.54]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.947, -1.447] < (-1.447, 0.0465] < (0.0465, 1.54] < (1.54, 3.034]]

- The Categorical object returned by cut can be passed directly to groupby.

In [73]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),'count': group.count(), 'mean': group.mean()}

In [74]:
grouped=frame.data2.groupby(quartiles)

In [75]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.947, -1.447]",-2.243828,1.838557,68.0,-0.063585
"(-1.447, 0.0465]",-2.798383,2.897189,431.0,-0.036014
"(0.0465, 1.54]",-3.222354,2.946356,435.0,0.023129
"(1.54, 3.034]",-2.826403,2.545399,66.0,-0.060983


In [76]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [77]:
grouped = frame.data2.groupby(grouping)

In [78]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.243828,2.897189,100.0,-0.043043
1,-2.798383,2.543278,100.0,0.085012
2,-2.507177,1.846160,100.0,-0.191204
3,-2.156602,1.809412,100.0,-0.043777
4,-2.278568,2.472857,100.0,0.006948
5,-2.000089,2.224564,100.0,0.153227
6,-3.222354,2.946356,100.0,-0.017287
7,-2.706108,2.273854,100.0,0.029700
8,-2.718477,2.574014,100.0,-0.024464


### Example: Filling Missing Values with Group-Specific Values

In [79]:
s = pd.Series(np.random.randn(6))

In [80]:
s[::2]=np.nan

In [81]:
s

0         NaN
1    0.236069
2         NaN
3    0.640678
4         NaN
5   -0.678089
dtype: float64

In [83]:
s.fillna(s.mean())

0    0.066219
1    0.236069
2    0.066219
3    0.640678
4    0.066219
5   -0.678089
dtype: float64

- Suppose you need the fill value to vary by group.

In [84]:
states = ['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']

In [85]:
group_key = ['East'] * 4 + ['West'] * 4

In [86]:
data=pd.Series(np.random.randn(8), index=states)

In [87]:
data

Ohio          0.919292
New York      2.790135
Vermont       1.526450
Florida      -0.778741
Oregon       -0.114397
Nevada        0.751615
California   -0.599743
Idaho        -0.035540
dtype: float64

In [88]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [89]:
data.groupby(group_key).mean()

East    0.976896
West   -0.357070
dtype: float64

In [90]:
fill_mean=lambda g:g.fillna(g.mean())

In [91]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.919292
New York      2.790135
Vermont       0.976896
Florida      -0.778741
Oregon       -0.114397
Nevada       -0.357070
California   -0.599743
Idaho        -0.357070
dtype: float64

### Example: Random Sampling and Permutation

In [93]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [94]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [95]:
def draw(deck, n=5):
    return deck.sample(n)

In [96]:
draw(deck)

8S      8
KH     10
10C    10
4H      4
2C      2
dtype: int64

In [97]:
get_suit = lambda card: card[-1]

In [98]:
get_suit

<function __main__.<lambda>(card)>

In [100]:
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   KC    10
D  3D     3
   QD    10
H  4H     4
   QH    10
S  2S     2
   KS    10
dtype: int64

In [101]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

KC     10
4C      4
2D      2
7D      7
QH     10
10H    10
6S      6
QS     10
dtype: int64

### Example: Group Weighted Average and Correlation

In [102]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
.....: 'b', 'b', 'b', 'b'],
.....: 'data': np.random.randn(8),
.....: 'weights': np.random.rand(8)})

In [103]:
df

,category,data,weights
0,a,1.442586,0.886156
1,a,-0.741035,0.842258
2,a,0.912599,0.653879
3,a,0.494421,0.631564
4,b,-0.287011,0.061907
5,b,1.167722,0.106020
6,b,0.651536,0.065373
7,b,0.050951,0.691511


In [104]:
grouped = df.groupby('category')

In [105]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [106]:
grouped.apply(get_wavg)

category
a    0.518671
b    0.198808
dtype: float64

In [107]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [108]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [109]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [110]:
rets=close_px.pct_change().dropna()

In [111]:
get_year=lambda x:x.year

In [112]:
by_year=rets.groupby(get_year)

In [113]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


### Example: Group-Wise Linear Regression

In [114]:
import statsmodels.api as sm

In [115]:
def regress(data, yvar, xvars):
    Y=data[yvar]
    X=data[xvars]
    X['intercept']=1.
    result=sm.OLS(Y,X).fit()
    return result.params

In [116]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation
- A pivot table is a data summarization tool frequently found in spreadsheet programs and other data analysis software.
- Pivot tables in Python with pandas are made possible through the groupby facility described in this chapter combined with reshape operations utilizing hierarchical indexing.

In [117]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [118]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

- margins=True. This has the effect of adding All row and column labels, with corresponding values being the group statistics for all the data within a single tier

In [120]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803